# Perception Asymmetry Feedback Loop: Opinion Bubble Stability

This notebook demonstrates the **Asymmetric Bounded-Confidence Agent-Based Model** that investigates how perception asymmetry affects opinion bubble stability in social networks.

## Key Hypothesis
Higher perception asymmetry (alpha) leads to more stable opinion bubbles because agents discount outgroup signals, preventing cluster merging.

## Method Overview
- **Baseline**: Standard bounded-confidence model (alpha=0, no asymmetry)
- **Method**: Asymmetric bounded-confidence model (alpha>0, agents discount outgroup signals by factor (1-alpha))

## Setup and Data Loading

In [ ]:
# Install dependencies if needed
# !pip install pandas matplotlib seaborn numpy scipy

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

In [ ]:
# Load data with fallback: try GitHub URL first, fall back to local file
GITHUB_DATA_URL = "https://raw.githubusercontent.com/AMGrobelnik/ai-invention-e59d7b-perception-asymmetry-feedback-loop-how-d/main/exp_exec_iter2_idx4/demo/demo_data.json"
LOCAL_DATA_PATH = "demo_data.json"

def load_data():
    """Load demo data from GitHub URL with local fallback."""
    try:
        # Try loading from GitHub (works in Colab after deployment)
        import urllib.request
        with urllib.request.urlopen(GITHUB_DATA_URL, timeout=5) as response:
            data = json.loads(response.read().decode())
            print(f"Loaded data from GitHub URL")
            return data
    except Exception as e:
        print(f"GitHub URL not available ({e}), falling back to local file...")
        # Fall back to local file (works during development)
        with open(LOCAL_DATA_PATH, 'r') as f:
            data = json.load(f)
            print(f"Loaded data from local file: {LOCAL_DATA_PATH}")
            return data

data = load_data()
print(f"\nLoaded {len(data['examples'])} examples")

## Data Exploration

In [ ]:
# Parse examples into a structured DataFrame
records = []
for ex in data['examples']:
    baseline = json.loads(ex['predict_baseline'])
    method = json.loads(ex['predict_method'])
    
    records.append({
        'seed': ex['context']['seed'],
        'method_alpha': ex['context']['method_alpha'],
        'baseline_alpha': ex['context']['baseline_alpha'],
        # Baseline metrics
        'baseline_cluster_count': baseline['final_cluster_count'],
        'baseline_lifetime': baseline['cluster_lifetime_mean'],
        'baseline_variance': baseline['final_variance'],
        'baseline_convergence': baseline['convergence_time'],
        # Method metrics
        'method_cluster_count': method['final_cluster_count'],
        'method_lifetime': method['cluster_lifetime_mean'],
        'method_variance': method['final_variance'],
        'method_convergence': method['convergence_time'],
        # Statistical context
        'correlation_r': ex['context']['correlation_r'],
        'correlation_p': ex['context']['correlation_p'],
        't_statistic': ex['context']['t_statistic'],
        't_pvalue': ex['context']['t_pvalue'],
    })

df = pd.DataFrame(records)
print(f"DataFrame shape: {df.shape}")
df.head()

In [ ]:
# Summary statistics
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
print(f"\nUnique alpha values tested: {sorted(df['method_alpha'].unique())}")
print(f"Number of seeds: {df['seed'].nunique()}")
print(f"\nBaseline (alpha=0) cluster lifetime mean: {df['baseline_lifetime'].mean():.2f}")
print(f"Method (alpha>0) cluster lifetime mean: {df['method_lifetime'].mean():.2f}")
print(f"\nStatistical significance:")
print(f"  - Correlation (alpha vs lifetime): r={df['correlation_r'].iloc[0]:.3f}, p={df['correlation_p'].iloc[0]:.2e}")
print(f"  - T-test (baseline vs method): t={df['t_statistic'].iloc[0]:.3f}, p={df['t_pvalue'].iloc[0]:.4f}")

## Visualization: Effect of Perception Asymmetry on Cluster Lifetime

In [ ]:
# Plot 1: Cluster Lifetime vs Alpha
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Scatter plot with trend line
ax1 = axes[0]
ax1.scatter(df['method_alpha'], df['method_lifetime'], alpha=0.7, s=80, c='steelblue', edgecolor='white', linewidth=0.5)

# Add baseline reference line
baseline_mean = df['baseline_lifetime'].mean()
ax1.axhline(y=baseline_mean, color='red', linestyle='--', linewidth=2, label=f'Baseline (alpha=0): {baseline_mean:.1f}')

# Fit linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(df['method_alpha'], df['method_lifetime'])
x_line = np.linspace(df['method_alpha'].min(), df['method_alpha'].max(), 100)
y_line = slope * x_line + intercept
ax1.plot(x_line, y_line, 'g-', linewidth=2, label=f'Trend (r={r_value:.2f}, p={p_value:.3f})')

ax1.set_xlabel('Perception Asymmetry (alpha)', fontsize=12)
ax1.set_ylabel('Cluster Lifetime (Mean)', fontsize=12)
ax1.set_title('Effect of Perception Asymmetry on Opinion Bubble Stability', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Right: Box plot by alpha groups
ax2 = axes[1]
df['alpha_group'] = pd.cut(df['method_alpha'], bins=[0, 0.4, 0.8, 1.2, 1.5], labels=['Low (0.1-0.4)', 'Medium (0.4-0.8)', 'High (0.8-1.2)', 'Very High (>1.2)'])
df.boxplot(column='method_lifetime', by='alpha_group', ax=ax2, grid=False)
ax2.axhline(y=baseline_mean, color='red', linestyle='--', linewidth=2, label='Baseline')
ax2.set_xlabel('Alpha Group', fontsize=12)
ax2.set_ylabel('Cluster Lifetime', fontsize=12)
ax2.set_title('Cluster Lifetime by Asymmetry Level', fontsize=14)
plt.suptitle('')

plt.tight_layout()
plt.show()

## Visualization: Method vs Baseline Comparison

In [ ]:
# Plot 2: Direct comparison of Method vs Baseline
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Prepare comparison data
metrics = ['lifetime', 'cluster_count', 'convergence']
titles = ['Cluster Lifetime', 'Final Cluster Count', 'Convergence Time']
colors = ['#2ecc71', '#e74c3c']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[i]
    
    baseline_vals = df[f'baseline_{metric}'].values
    method_vals = df[f'method_{metric}'].values
    
    x = np.arange(len(df))
    width = 0.35
    
    ax.bar(x - width/2, baseline_vals, width, label='Baseline (alpha=0)', color=colors[1], alpha=0.7)
    ax.bar(x + width/2, method_vals, width, label='Method (alpha>0)', color=colors[0], alpha=0.7)
    
    ax.set_xlabel('Sample Index', fontsize=11)
    ax.set_ylabel(title, fontsize=11)
    ax.set_title(f'{title}: Method vs Baseline', fontsize=12)
    ax.legend(loc='upper right')
    ax.set_xticks(x[::3])

plt.tight_layout()
plt.show()

## Statistical Analysis

In [ ]:
# Detailed statistical comparison
print("=" * 60)
print("STATISTICAL ANALYSIS")
print("=" * 60)

# T-test for cluster lifetime
t_stat, p_val = stats.ttest_ind(df['baseline_lifetime'], df['method_lifetime'])
print(f"\nT-test (Cluster Lifetime - Method vs Baseline):")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_val:.4f}")
print(f"  Significant (p<0.05): {'Yes' if p_val < 0.05 else 'No'}")

# Effect size (Cohen's d)
pooled_std = np.sqrt((df['baseline_lifetime'].std()**2 + df['method_lifetime'].std()**2) / 2)
cohens_d = (df['method_lifetime'].mean() - df['baseline_lifetime'].mean()) / pooled_std
print(f"\nEffect Size (Cohen's d): {cohens_d:.3f}")
if abs(cohens_d) < 0.2:
    effect_interpretation = "negligible"
elif abs(cohens_d) < 0.5:
    effect_interpretation = "small"
elif abs(cohens_d) < 0.8:
    effect_interpretation = "medium"
else:
    effect_interpretation = "large"
print(f"  Interpretation: {effect_interpretation} effect")

# Correlation between alpha and cluster lifetime
r, p = stats.pearsonr(df['method_alpha'], df['method_lifetime'])
print(f"\nCorrelation (Alpha vs Method Lifetime):")
print(f"  Pearson r: {r:.4f}")
print(f"  p-value: {p:.4f}")
print(f"  Significant (p<0.05): {'Yes' if p < 0.05 else 'No'}")

## Results Summary

In [ ]:
# Final summary
print("=" * 60)
print("KEY FINDINGS")
print("=" * 60)

avg_baseline_lifetime = df['baseline_lifetime'].mean()
avg_method_lifetime = df['method_lifetime'].mean()
improvement = ((avg_method_lifetime - avg_baseline_lifetime) / avg_baseline_lifetime) * 100

print(f"\n1. Average Cluster Lifetime:")
print(f"   - Baseline (no asymmetry): {avg_baseline_lifetime:.2f} timesteps")
print(f"   - Method (with asymmetry): {avg_method_lifetime:.2f} timesteps")
print(f"   - Improvement: {improvement:+.1f}%")

print(f"\n2. Hypothesis Support:")
if improvement > 0 and p_val < 0.05:
    print("   CONFIRMED: Higher perception asymmetry leads to MORE stable opinion bubbles.")
    print("   The effect is statistically significant.")
elif improvement > 0:
    print("   PARTIALLY CONFIRMED: Trend suggests more stable bubbles with asymmetry,")
    print("   but more data may be needed for statistical significance.")
else:
    print("   NOT CONFIRMED: Results do not support the hypothesis.")

print(f"\n3. Mechanism:")
print("   Agents with higher alpha discount outgroup signals by factor (1-alpha),")
print("   reducing cross-cluster influence and preventing cluster merging.")

## Example Data Inspection

In [ ]:
# Show a sample example from the data
print("Sample Example from Dataset:")
print("=" * 60)
sample = data['examples'][0]
print(f"\nInput: {sample['input']}")
print(f"\nOutput (Hypothesis): {sample['output']}")
print(f"\nMethod: {sample['method']}")
print(f"\nBaseline Prediction:")
print(json.dumps(json.loads(sample['predict_baseline']), indent=2))
print(f"\nMethod Prediction:")
print(json.dumps(json.loads(sample['predict_method']), indent=2))